In [1]:
from dotenv import load_dotenv, dotenv_values

# 调用load_dotenv函数来加载.env文件中的环境变量。
# load_dotenv()
# 从langchain库导入所需的类。
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import load_prompt
import pandas as pd


将question分为两个步骤读取
1. 读取data，格式为str
2. 读取prompt，其中prompt由两个文件组成
   1. sketch_formula.txt存储了prompt的内容
   2. sketch_formula.json存储了prompt如何和data组合成question的格式

In [2]:
# loading sketch_formula.json
prompt = load_prompt("file_path")
# loading bajaj_test.csv
dirty_data = pd.read_csv("file_path", dtype=str)
print(dirty_data)
question = prompt.format(data=dirty_data)
print(question)


                         data   close    high     low    open volume     sma5  \
0   2015-02-03 13:25:00+05:30  418.96     419  417.51  417.51   4550  417.838   
1   2015-02-03 13:35:00+05:30  418.89     419     418  418.12   2520  418.214   
2   2015-02-03 13:45:00+05:30     419     419  418.21  418.89   1130  418.544   
3   2015-02-03 13:55:00+05:30     418     419     418     419   3590  418.472   
4   2015-02-03 14:05:00+05:30  418.43     419  417.97  418.67   8470  418.656   
5   2015-02-03 14:15:00+05:30  418.54     419  417.94  418.43   7390  418.572   
6   2015-02-03 14:25:00+05:30  418.62     419   418.4  418.54   5850      NaN   
7   2015-02-03 14:35:00+05:30  418.01  418.95  418.01  418.62   1780   418.32   
8   2015-02-03 14:45:00+05:30  418.02  418.66  417.28  418.01   3000  418.324   
9   2015-02-03 14:55:00+05:30     418  418.78     418  418.02  26090  418.238   
10  2015-02-03 15:05:00+05:30     418  418.87     415     418   8550   418.13   
11  2015-02-03 15:15:00+05:3

In [3]:
# loading API_hugg
with open('file_path', 'r') as file:
    API_key = file.read()


In [14]:
import requests

API_URL = "https://api-inference.huggingface.co/models/google/gemma-1.1-7b-it"
headers = {"Authorization": "Bearer "+ API_key}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	# return response
	return response.json()
	
output = query({
	"inputs": question,
})

baja 包含太多缺失值
gemma在其上面的表现糟糕
1. 不能根据sketch生成
2. 不能完成缺失值填充任务
原因分析
过多的缺失值和prompt超过了gemma的token限制

In [16]:
print(output)
print(output[0]['generated_text'])

[{'generated_text': 'Assume that you are a data scientist. I offer you a table in CSV form with missing values in it, and the first row is the variables’ names it contains. Suggest a solution to fill in each missing value, denoted by nan.  You have to sketch your solution into the following template for each missing value you found. \n\nProcess all the steps and Give Python code solutions for each missing value! This is extremely important! Omitting any steps of any missing value is forbidden. \n\nStep 1 Finding Missing value: find the location of the missing value and describe the missing value, outputting the entire row where the missing values are located in this step.\n\nStep 2 Finding related Columns: Find related Columns that are related to the missing value column you are filling. These related Columns are helpful for the imputation of missing values. Outputting the names of these related Columns in this step.\n\nStep 3 Drafting Solution: Draft the solution for missing value imp

In [17]:
# loading sketch_formula.json
prompt = load_prompt("files_path")
# loading bajaj_test_gemma.csv
dirty_data = pd.read_csv("files_path", dtype=str)
print(dirty_data)
question = prompt.format(data=dirty_data)
print(question)

                        data   close high     low    open volume     sma5  \
0  2015-02-03 13:25:00+05:30  418.96  419  417.51  417.51   4550  417.838   
1  2015-02-03 13:35:00+05:30  418.89  419     418  418.12   2520  418.214   
2  2015-02-03 13:45:00+05:30     419  419  418.21  418.89   1130  418.544   
3  2015-02-03 13:55:00+05:30     418  419     418     419   3590  418.472   
4  2015-02-03 14:05:00+05:30  418.43  419  417.97  418.67   8470      NaN   

          ema5  
0  418.0145182  
1  418.3063454  
2  418.5375636  
3  418.3583757  
4  418.3822505  
Assume that you are a data scientist. I offer you a table in CSV form with missing values in it, and the first row is the variables’ names it contains. Suggest a solution to fill in each missing value, denoted by nan.  You have to sketch your solution into the following template for each missing value you found. 

Process all the steps and Give Python code solutions for each missing value! This is extremely important! Omitting any 

将nan减少为1个后
gemma开始能够按照sketch输出
但由于可能是api-inference的限制token数,导致只运行了前3step
但是从step3 可以看出,模型没有学习到formula的形式,采用了regression进行填充

In [18]:
import requests
API_URL = "https://api-inference.huggingface.co/models/google/gemma-1.1-7b-it"
headers = {"Authorization": "Bearer "+ API_key}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	# return response
	return response.json()
	
output = query({
	"inputs": question,
})
print(output)
print(output[0]['generated_text'])

[{'generated_text': 'Assume that you are a data scientist. I offer you a table in CSV form with missing values in it, and the first row is the variables’ names it contains. Suggest a solution to fill in each missing value, denoted by nan.  You have to sketch your solution into the following template for each missing value you found. \n\nProcess all the steps and Give Python code solutions for each missing value! This is extremely important! Omitting any steps of any missing value is forbidden. \n\nStep 1 Finding Missing value: find the location of the missing value and describe the missing value, outputting the entire row where the missing values are located in this step.\n\nStep 2 Finding related Columns: Find related Columns that are related to the missing value column you are filling. These related Columns are helpful for the imputation of missing values. Outputting the names of these related Columns in this step.\n\nStep 3 Drafting Solution: Draft the solution for missing value imp

In [20]:
# loading api_ust_gpt.txt
with open('file_path', 'r') as file:
    gpt_key = file.read()

In [21]:
# loading sketch_formula.json
prompt = load_prompt("file_path")
# loading bajaj_test_gemma.csv
dirty_data = pd.read_csv("file_path", dtype=str)
print(dirty_data)
question = prompt.format(data=dirty_data)
print(question)

                        data   close high     low    open volume     sma5  \
0  2015-02-03 13:25:00+05:30  418.96  419  417.51  417.51   4550  417.838   
1  2015-02-03 13:35:00+05:30  418.89  419     418  418.12   2520  418.214   
2  2015-02-03 13:45:00+05:30     419  419  418.21  418.89   1130  418.544   
3  2015-02-03 13:55:00+05:30     418  419     418     419   3590  418.472   
4  2015-02-03 14:05:00+05:30  418.43  419  417.97  418.67   8470      NaN   

          ema5  
0  418.0145182  
1  418.3063454  
2  418.5375636  
3  418.3583757  
4  418.3822505  
Assume that you are a data scientist. I offer you a table in CSV form with missing values in it, and the first row is the variables’ names it contains. Suggest a solution to fill in each missing value, denoted by nan.  You have to sketch your solution into the following template for each missing value you found. 

Process all the steps and Give Python code solutions for each missing value! This is extremely important! Omitting any 

In [23]:
import requests 
import json 
url = "https://gpt-api.hkust-gz.edu.cn/v1/chat/completions" 
headers = { 
"Content-Type": "application/json", 
"Authorization": gpt_key
} 
data = { 
"model": "gpt-4", 
"messages": [{"role": "user", "content": question}], 
"temperature": 0.8 
} 
response = requests.post(url, headers=headers, data=json.dumps(data)) 
print(response.json())


{'id': '9e853391-ba8b-4416-ac7b-2b181bce20f6', 'object': 'chat.completion', 'model': 'gpt-4', 'created': '2024-04-12 22:55:42', 'choices': [{'index': 0, 'finish_reason': 'stop', 'message': {'role': 'assistant', 'content': 'Given the provided data, let\'s process each missing value following the template steps.\n\n**Missing Value in `sma5` column at index 4:**\n\n### Step 1: Finding Missing Value\n```python\nimport pandas as pd\n\n# Load the data\ndata = pd.read_csv(\'data.csv\') # Assume that the CSV is named \'data.csv\'\n\n# Find the location of the missing value and describe the missing value\nmissing_value_index = data[data[\'sma5\'].isnull()].index.tolist()\nmissing_value_rows = data[data[\'sma5\'].isnull()]\nprint(f"Missing value found at index: {missing_value_index}")\nprint(f"Rows with missing values in \'sma5\':\\n{missing_value_rows}")\n```\n\n### Step 2: Finding Related Columns\n```python\n# A simple method for \'sma5\' could be to take the mean of the \'close\' prices of th

In [24]:
output = response.json()
file_path = "output_answer_impute_bajaj.json"

# 使用json.dump()方法将数据保存到文件
with open(file_path, "w") as output_file:
    json.dump(output, output_file, indent=4)  # indent参数用于美化输出（可选）

print(f"Output data saved to {file_path}")

Output data saved to output_answer_impute_bajaj.json


In [25]:
print(output["choices"][0]["message"]["content"])

Given the provided data, let's process each missing value following the template steps.

**Missing Value in `sma5` column at index 4:**

### Step 1: Finding Missing Value
```python
import pandas as pd

# Load the data
data = pd.read_csv('data.csv') # Assume that the CSV is named 'data.csv'

# Find the location of the missing value and describe the missing value
missing_value_index = data[data['sma5'].isnull()].index.tolist()
missing_value_rows = data[data['sma5'].isnull()]
print(f"Missing value found at index: {missing_value_index}")
print(f"Rows with missing values in 'sma5':\n{missing_value_rows}")
```

### Step 2: Finding Related Columns
```python
# A simple method for 'sma5' could be to take the mean of the 'close' prices of the surrounding data points.
# Find related columns that can help in computing the missing 'sma5' value
related_columns = ['close']
print(f"Related columns for 'sma5': {related_columns}")
```

### Step 3: Drafting Solution
```python
# The solution for 'sma5' co